In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/yelp-dataset/Dataset_User_Agreement.pdf
/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_checkin.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json


In [3]:
#2 - Converting JSON data to a Pandas DataFrame and further exploration

import pandas as pd
import json

def load_json_to_df(file_path):
    data_file = open(file_path)
    data = []
    for line in data_file:
        data.append(json.loads(line))
    return pd.DataFrame(data)

# Now let's load each JSON data file
business_df = load_json_to_df('/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json')
print("business_df loaded.")
checkin_df = load_json_to_df('/kaggle/input/yelp-dataset/yelp_academic_dataset_checkin.json')
print("checkin_df loaded.")
review_df = load_json_to_df('/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json')
print("review_df loaded.")
tip_df = load_json_to_df('/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json')
print("tip_df loaded.")
user_df = load_json_to_df('/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json')
print("user_df loaded.")

business_df loaded.
checkin_df loaded.
review_df loaded.
tip_df loaded.
user_df loaded.


# Observe ,Explore and Deal with Data

In [3]:
# Explore the structure
# df.info()
# df.head()
# df.shape

# Explore the time period
# df['date_column'] = pd.to_datetime(df['date_column'])
# start_date = df['date_column'].min()
# end_date = df['date_column'].max()
# print(f"Time period covered: {start_date} to {end_date}")

# Scope of the database
# df['your_column'].nunique() -- count unique values in a column
# df.isnull().sum() -- identify missing value
# df.groupby('your_categorical_column').agg({'your_numeric_column': ['mean', 'sum']})

## Business_df 

In [52]:
business_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1


In [5]:
# Explore the structure
business_df.info()
business_df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   150346 non-null  object 
 1   name          150346 non-null  object 
 2   address       150346 non-null  object 
 3   city          150346 non-null  object 
 4   state         150346 non-null  object 
 5   postal_code   150346 non-null  object 
 6   latitude      150346 non-null  float64
 7   longitude     150346 non-null  float64
 8   stars         150346 non-null  float64
 9   review_count  150346 non-null  int64  
 10  is_open       150346 non-null  int64  
 11  attributes    136602 non-null  object 
 12  categories    150243 non-null  object 
 13  hours         127123 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 16.1+ MB


(150346, 14)

In [6]:
business_df.isnull().sum() # identify missing value

business_id         0
name                0
address             0
city                0
state               0
postal_code         0
latitude            0
longitude           0
stars               0
review_count        0
is_open             0
attributes      13744
categories        103
hours           23223
dtype: int64

### categories_df -- from business_df (v)

In [7]:
# Create a new DataFrame by copying the necessary columns from business_df
cat_df = business_df[['business_id', 'categories']].copy()

# Remove unwanted characters such as brackets and quotes using regular expressions
cat_df['categories'] = cat_df['categories'].str.replace(r"[\[\]']", '', regex=True)

# Split the categories by comma followed by a space
cat_df['categories'] = cat_df['categories'].astype(str).str.split(', ')

# Explode the DataFrame on the 'categories' column to separate each category
cat_df_exploded = cat_df.explode('categories')

# Drop any rows where categories are NaN after exploding
cat_df_exploded = cat_df_exploded.dropna(subset=['categories'])

# Create a mapping from each unique category to a unique integer ID
categories_unique = cat_df_exploded['categories'].unique()
category_to_id = {category: idx + 1 for idx, category in enumerate(categories_unique)}

# Map the categories in the DataFrame to their unique integer IDs
cat_df_exploded['cat_id'] = cat_df_exploded['categories'].map(category_to_id)

# Create the final DataFrame with only the required columns
categories_df = cat_df_exploded[['business_id', 'cat_id', 'categories']].rename(columns={'categories': 'category'})

# Reset the index of the final DataFrame
categories_df.reset_index(drop=True, inplace=True)

# Display the first few rows of the final DataFrame
categories_df.head(30)

,business_id,cat_id,category
0,Pns2l4eNsfO8kk83dixA6A,1,Doctors
1,Pns2l4eNsfO8kk83dixA6A,2,Traditional Chinese Medicine
2,Pns2l4eNsfO8kk83dixA6A,3,Naturopathic/Holistic
3,Pns2l4eNsfO8kk83dixA6A,4,Acupuncture
4,Pns2l4eNsfO8kk83dixA6A,5,Health & Medical
5,Pns2l4eNsfO8kk83dixA6A,6,Nutritionists
6,mpf3x-BjTdTEA3yCZrAYPw,7,Shipping Centers
7,mpf3x-BjTdTEA3yCZrAYPw,8,Local Services
8,mpf3x-BjTdTEA3yCZrAYPw,9,Notaries
9,mpf3x-BjTdTEA3yCZrAYPw,10,Mailbox Centers


In [8]:
business_df = business_df.drop('categories', axis=1)
business_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


### hour_df -- from business_df (v)

In [9]:
# Initialize an empty list to store normalized business hours data
normalized_hours = []

# Iterate through each row of business data
for _, row in business_df.iterrows():
    business_id = row['business_id']
    hours_dict = row['hours']
    
    # Check if the 'hours' field is not empty
    if hours_dict is not None:
        # Iterate through each set of business hours
        for day, hours_range in hours_dict.items():
            start, end = hours_range.split('-')
            
            # Calculate duration in hours and minutes
            start_hour, start_minute = map(int, start.split(':'))
            end_hour, end_minute = map(int, end.split(':'))
            duration_minutes = ((end_hour * 60 + end_minute) - (start_hour * 60 + start_minute)) % (24 * 60)
            duration_hours, duration_minutes = divmod(duration_minutes, 60)
            duration_str = f"{duration_hours}:{duration_minutes:02d}"
            
            normalized_hours.append({
                'business_id': business_id,
                'day_of_week': day,
                'start_hour': start,
                'end_hour': end,
                'duration': duration_str  # Added duration in hours:minutes format
            })

# Create a new DataFrame from the normalized_hours list
hours_df = pd.DataFrame(normalized_hours)

In [10]:
# check the new table
hours_df.head(30)

,business_id,day_of_week,start_hour,end_hour,duration
0,mpf3x-BjTdTEA3yCZrAYPw,Monday,0:0,0:0,0:00
1,mpf3x-BjTdTEA3yCZrAYPw,Tuesday,8:0,18:30,10:30
2,mpf3x-BjTdTEA3yCZrAYPw,Wednesday,8:0,18:30,10:30
3,mpf3x-BjTdTEA3yCZrAYPw,Thursday,8:0,18:30,10:30
4,mpf3x-BjTdTEA3yCZrAYPw,Friday,8:0,18:30,10:30
5,mpf3x-BjTdTEA3yCZrAYPw,Saturday,8:0,14:0,6:00
6,tUFrWirKiKi_TAnsVWINQQ,Monday,8:0,22:0,14:00
7,tUFrWirKiKi_TAnsVWINQQ,Tuesday,8:0,22:0,14:00
8,tUFrWirKiKi_TAnsVWINQQ,Wednesday,8:0,22:0,14:00
9,tUFrWirKiKi_TAnsVWINQQ,Thursday,8:0,22:0,14:00


In [11]:
# drop "hours" from business_df
business_df = business_df.drop('hours', axis=1)
business_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'}
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'}
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc..."


### attribute_df -- from business_df(v)

In [12]:
import pandas as pd

# Replace 'None' with an empty dictionary
business_df['attributes'] = business_df['attributes'].apply(lambda x: x if isinstance(x, dict) else {})

# Normalize the 'attributes' column of dictionaries into a separate DataFrame
attributes_df = pd.json_normalize(business_df['attributes'])

# Merge this DataFrame with the 'business_id'
attributes_df = business_df[['business_id']].join(attributes_df)

# Replace string 'True' and 'False' with actual boolean values True and False
attributes_df = attributes_df.applymap(lambda x: True if x == 'True' else False if x == 'False' else x)

# Make sure all the data types are correct after the replacement
attributes_df = attributes_df.infer_objects()

/tmp/ipykernel_117/2589787604.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  attributes_df = attributes_df.applymap(lambda x: True if x == 'True' else False if x == 'False' else x)


In [13]:
attributes_df.shape
# pd.set_option('display.max_colwidth', None)  # Set to None to not limit column widths
# pd.set_option('display.max_columns', None)  # Set to None to display all columns
# attributes_df.iloc[:,10:20]
# attributes_df.head()

(150346, 40)

In [14]:
# Data wrangling for attributes_df.iloc[:,10]

import ast  # For converting string representation of dictionaries into actual dictionaries

# Function to convert a string representation of a dictionary into an actual dictionary
def parse_parking(row):
    try:
        return ast.literal_eval(row)  # Parse string to dictionary
    except (ValueError, SyntaxError):
        return {}  # Return an empty dictionary in case of parsing error

# Convert each value in 'BusinessParking' column to a dictionary
attributes_df['BusinessParking'] = attributes_df['BusinessParking'].apply(parse_parking)

# Get the index of the 'BusinessParking' column
parking_col_index = attributes_df.columns.get_loc('BusinessParking')

# Expand the dictionary into multiple columns
expanded_columns = attributes_df['BusinessParking'].apply(pd.Series)

# Remove the original 'BusinessParking' column
attributes_df = attributes_df.drop(['BusinessParking'], axis=1)

# Insert the expanded columns back into the DataFrame at the original 'BusinessParking' column's position
# and rename them with a prefix 'BusinessParking_'
for col in expanded_columns.columns:
    new_col_name = f'BusinessParking_{col}'  # Add prefix to the column name
    attributes_df.insert(loc=parking_col_index, column=new_col_name, value=expanded_columns[col])
    parking_col_index += 1  # Increment the position for the next column to be inserted

In [15]:
# Data Wrangling for other data in attributes_df

# Replace NaN values with False for boolean columns
bool_cols = ['ByAppointmentOnly', 'BusinessAcceptsCreditCards', 'BikeParking', 
             'CoatCheck', 'RestaurantsTakeOut', 'RestaurantsDelivery', 'Caters']
attributes_df[bool_cols] = attributes_df[bool_cols].fillna(False)

# Standardize 'WiFi' column to boolean where 'u'no'' is False and 'u'free'' is True
attributes_df['WiFi'] = attributes_df['WiFi'].replace({"u'no'": False, "u'free'": True})

# Convert 'RestaurantsPriceRange2' to numeric, leaving NaN as is to represent unknown
attributes_df['RestaurantsPriceRange2'] = pd.to_numeric(attributes_df['RestaurantsPriceRange2'], errors='coerce')

attributes_df['Alcohol'] = attributes_df['Alcohol'].replace({"u'none'": 'none', "u'full_bar'": 'full_bar', "u'beer_and_wine'": 'beer_and_wine'})

attributes_df['RestaurantsAttire'] = attributes_df['RestaurantsAttire'].apply(lambda x: False if x == "u'none'" else True)

# Replace 'None' with False for boolean columns
attributes_df = attributes_df.replace({None: False})

attributes_df = attributes_df.replace({r"u'([^']*)'": r"\1"}, regex=True)

# Make sure all the data types are correct after the replacements
attributes_df = attributes_df.infer_objects()

/tmp/ipykernel_117/3572851523.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  attributes_df = attributes_df.replace({None: False})


In [16]:
# check the result if the wrangling works well
attributes_df.shape
attributes_df.head(100)

,business_id,ByAppointmentOnly,BusinessAcceptsCreditCards,BikeParking,RestaurantsPriceRange2,CoatCheck,RestaurantsTakeOut,RestaurantsDelivery,Caters,WiFi,...,AcceptsInsurance,BestNights,BYOB,Corkage,BYOBCorkage,HairSpecializesIn,Open24Hours,RestaurantsCounterService,AgesAllowed,DietaryRestrictions
0,Pns2l4eNsfO8kk83dixA6A,True,False,False,NaN,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,mpf3x-BjTdTEA3yCZrAYPw,False,True,False,NaN,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,tUFrWirKiKi_TAnsVWINQQ,False,True,True,2.0,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,MTSW4McQd7CbVtyjqoe9mw,False,False,True,1.0,False,True,False,True,True,...,False,False,False,False,False,False,False,False,False,False
4,mWMc6_wTdE0EUBKIGXDVfA,False,True,True,NaN,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,pEm4xNCk8d0TF6A1gi-WDQ,False,True,False,NaN,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
96,J_ksUDPpzPwfTGtI4zTRnQ,False,True,False,NaN,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
97,knQ4vIgx-r85kjlWVVjcpQ,False,True,True,NaN,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
98,U2Tzy6J9aAAg9pzXsgdkvg,False,False,False,NaN,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [17]:
# drop "attributes" from business_df
business_df = business_df.drop('attributes', axis=1)
business_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1


## checkin_df

In [18]:
checkin_df.head()

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012..."
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014..."


In [19]:
checkin_df.info()
checkin_df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131930 entries, 0 to 131929
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   business_id  131930 non-null  object
 1   date         131930 non-null  object
dtypes: object(2)
memory usage: 2.0+ MB


(131930, 2)

In [20]:
checkin_df.isnull().sum() # identify missing value

business_id    0
date           0
dtype: int64

### chechin_df_split -- from checkin_df (v)

In [21]:
# Create a new DataFrame by copying the 'date' column from checkin_df
chec_df = checkin_df.copy()

# Split the dates on each row by comma followed by a space
chec_df['date'] = chec_df['date'].str.split(', ')

# Explode the new DataFrame on the 'date' column to separate each date
checkin_df_split = chec_df.explode('date')

# Reset the index of the new exploded DataFrame
checkin_df_split.reset_index(drop=True, inplace=True)

# Display the first 20 rows of the new DataFrame
checkin_df_split.head(20)

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,2020-03-13 21:10:56
1,---kPU91CF4Lq2-WlRu9Lw,2020-06-02 22:18:06
2,---kPU91CF4Lq2-WlRu9Lw,2020-07-24 22:42:27
3,---kPU91CF4Lq2-WlRu9Lw,2020-10-24 21:36:13
4,---kPU91CF4Lq2-WlRu9Lw,2020-12-09 21:23:33
5,---kPU91CF4Lq2-WlRu9Lw,2021-01-20 17:34:57
6,---kPU91CF4Lq2-WlRu9Lw,2021-04-30 21:02:03
7,---kPU91CF4Lq2-WlRu9Lw,2021-05-25 21:16:54
8,---kPU91CF4Lq2-WlRu9Lw,2021-08-06 21:08:08
9,---kPU91CF4Lq2-WlRu9Lw,2021-10-02 15:15:42


In [22]:
# explore the time period of chekin
start_date = checkin_df_split['date'].min()
end_date = checkin_df_split['date'].max()
print("Minimum Date:", start_date)
print("Maximum Date:", end_date)

Minimum Date: 2009-12-30 02:53:27
Maximum Date: 2022-01-19 16:48:37


In [23]:
# explore count of chekin
total_checkins = checkin_df_split.shape[0]
print(f"Total Checkin: {total_checkins}")

Total Checkin: 13356875


In [24]:
import pandas as pd

# If 'date' is not in datetime format, convert it first:
checkin_df_split['date'] = pd.to_datetime(checkin_df_split['date'])

# Add columns for year, month, day of the week, etc.
checkin_df_split['year'] = checkin_df_split['date'].dt.year
checkin_df_split['month'] = checkin_df_split['date'].dt.month
checkin_df_split['day_of_week'] = checkin_df_split['date'].dt.day_name()
checkin_df_split['hour_of_day'] = checkin_df_split['date'].dt.hour
checkin_df_split['week_of_year'] = checkin_df_split['date'].dt.isocalendar().week
checkin_df_split['is_weekend'] = checkin_df_split['date'].dt.dayofweek > 4

# Convert 'date' to datetime if it isn't already
checkin_df_split['date'] = pd.to_datetime(checkin_df_split['date'])

# Group by 'business_id' to aggregate data
grouped = checkin_df_split.groupby('business_id')

# Count total check-ins per business
checkin_counts = grouped['date'].count().rename('checkin_count')

# Count unique days per business
unique_days_count = grouped['date'].nunique().rename('unique_days_count')

# Calculate average check-ins per day the business had at least one check-in
avg_checkins_per_day = (grouped['date'].count() / grouped['date'].nunique()).rename('avg_checkins_per_day')

# Join these calculations back to the original dataframe
business_summary = pd.DataFrame({
    'checkin_count': checkin_counts,
    'unique_days_count': unique_days_count,
    'avg_checkins_per_day': avg_checkins_per_day
}).reset_index()

checkin_df_split = pd.merge(checkin_df_split, business_summary, on='business_id', how='left')

checkin_df_split.head(20)

,business_id,date,year,month,day_of_week,hour_of_day,week_of_year,is_weekend,checkin_count,unique_days_count,avg_checkins_per_day
0,---kPU91CF4Lq2-WlRu9Lw,2020-03-13 21:10:56,2020,3,Friday,21,11,False,11,11,1.0
1,---kPU91CF4Lq2-WlRu9Lw,2020-06-02 22:18:06,2020,6,Tuesday,22,23,False,11,11,1.0
2,---kPU91CF4Lq2-WlRu9Lw,2020-07-24 22:42:27,2020,7,Friday,22,30,False,11,11,1.0
3,---kPU91CF4Lq2-WlRu9Lw,2020-10-24 21:36:13,2020,10,Saturday,21,43,True,11,11,1.0
4,---kPU91CF4Lq2-WlRu9Lw,2020-12-09 21:23:33,2020,12,Wednesday,21,50,False,11,11,1.0
5,---kPU91CF4Lq2-WlRu9Lw,2021-01-20 17:34:57,2021,1,Wednesday,17,3,False,11,11,1.0
6,---kPU91CF4Lq2-WlRu9Lw,2021-04-30 21:02:03,2021,4,Friday,21,17,False,11,11,1.0
7,---kPU91CF4Lq2-WlRu9Lw,2021-05-25 21:16:54,2021,5,Tuesday,21,21,False,11,11,1.0
8,---kPU91CF4Lq2-WlRu9Lw,2021-08-06 21:08:08,2021,8,Friday,21,31,False,11,11,1.0
9,---kPU91CF4Lq2-WlRu9Lw,2021-10-02 15:15:42,2021,10,Saturday,15,39,True,11,11,1.0


In [25]:
# check if there's days with more than 1 checkin for a business_id, to know if we should preserve unique_days_count and avg_checkins_per_day
mean_avg_checkins_per_day = checkin_df_split['avg_checkins_per_day'].mean()
print(mean_avg_checkins_per_day)

# preserve v

1.0002681393556085


## review_df

In [26]:
review_df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [27]:
review_df.info()
review_df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6990280 entries, 0 to 6990279
Data columns (total 9 columns):
 #   Column       Dtype  
---  ------       -----  
 0   review_id    object 
 1   user_id      object 
 2   business_id  object 
 3   stars        float64
 4   useful       int64  
 5   funny        int64  
 6   cool         int64  
 7   text         object 
 8   date         object 
dtypes: float64(1), int64(3), object(5)
memory usage: 480.0+ MB


(6990280, 9)

In [28]:
# Explore the time period
review_df['date'] = pd.to_datetime(review_df['date'])
start_date = review_df['date'].min()
end_date = review_df['date'].max()
print(f"Time period covered: {start_date} to {end_date}")

Time period covered: 2005-02-16 03:23:22 to 2022-01-19 19:48:45


In [29]:
review_df.isnull().sum() # identify missing value

review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64

## tip_df

In [30]:
tip_df.head()

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0


In [31]:
tip_df.info()
tip_df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908915 entries, 0 to 908914
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_id           908915 non-null  object
 1   business_id       908915 non-null  object
 2   text              908915 non-null  object
 3   date              908915 non-null  object
 4   compliment_count  908915 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 34.7+ MB


(908915, 5)

In [32]:
# Explore the time period
tip_df['date'] = pd.to_datetime(tip_df['date'])
start_date = tip_df['date'].min()
end_date = tip_df['date'].max()
print(f"Time period covered: {start_date} to {end_date}")

Time period covered: 2009-04-16 13:11:49 to 2022-01-19 20:38:55


In [33]:
tip_df.isnull().sum() # identify missing value

user_id             0
business_id         0
text                0
date                0
compliment_count    0
dtype: int64

## user_df

In [4]:
# Restore display settings to default values
# pd.set_option('display.max_rows', 60)
# pd.set_option('display.max_columns', 20)

user_df.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,...,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"2009,2010,2011","enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",28,...,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,...,1,0,0,0,1,1,0,0,0,0


In [4]:
user_df.info()
user_df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1987897 entries, 0 to 1987896
Data columns (total 22 columns):
 #   Column              Dtype  
---  ------              -----  
 0   user_id             object 
 1   name                object 
 2   review_count        int64  
 3   yelping_since       object 
 4   useful              int64  
 5   funny               int64  
 6   cool                int64  
 7   elite               object 
 8   friends             object 
 9   fans                int64  
 10  average_stars       float64
 11  compliment_hot      int64  
 12  compliment_more     int64  
 13  compliment_profile  int64  
 14  compliment_cute     int64  
 15  compliment_list     int64  
 16  compliment_note     int64  
 17  compliment_plain    int64  
 18  compliment_cool     int64  
 19  compliment_funny    int64  
 20  compliment_writer   int64  
 21  compliment_photos   int64  
dtypes: float64(1), int64(16), object(5)
memory usage: 333.7+ MB


(1987897, 22)

In [5]:
user_df.isnull().sum() # identify missing value

user_id               0
name                  0
review_count          0
yelping_since         0
useful                0
funny                 0
cool                  0
elite                 0
friends               0
fans                  0
average_stars         0
compliment_hot        0
compliment_more       0
compliment_profile    0
compliment_cute       0
compliment_list       0
compliment_note       0
compliment_plain      0
compliment_cool       0
compliment_funny      0
compliment_writer     0
compliment_photos     0
dtype: int64

In [6]:
# check if there's data needed to deal with
# user_df.iloc[:,1:10]

## friend_df -- from user_df (v)

In [4]:
# seperate friend_df from user_df

# Define a function to process a chunk of the user DataFrame
def process_chunk(chunk):
    friend_data = []  # Initialize an empty list to hold the processed data
    
    for index, row in chunk.iterrows():
        user_id = row['user_id']
        friends = str(row['friends']).split(',')  # Ensure it's a string and split
        friend_data.extend([(user_id, friend.strip()) for friend in friends if friend.strip()])  # Strip spaces and check if not empty

    # Return a DataFrame constructed from the processed data
    return pd.DataFrame(friend_data, columns=['user_id', 'friend_id'])

# Initialize an empty DataFrame for the friend data
friend_df = pd.DataFrame(columns=['user_id', 'friend_id'])

# Define the chunk size
chunk_size = 50000  # Adjust this based on your memory constraints

# Process the user_df in chunks
for start in range(0, user_df.shape[0], chunk_size):
    end = min(start + chunk_size, user_df.shape[0])
    chunk = user_df.iloc[start:end]
    friend_df_chunk = process_chunk(chunk)
    
    friend_df = pd.concat([friend_df, friend_df_chunk], ignore_index=True)
    print(end)

50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000
700000
750000
800000
850000
900000
950000
1000000
1050000
1100000
1150000
1200000
1250000
1300000
1350000
1400000
1450000
1500000
1550000
1600000
1650000
1700000
1750000
1800000
1850000
1900000
1950000
1987897


In [5]:
friend_df.head()

,user_id,friend_id
0,qVc8ODYU5SZjKXVBgXdI7w,NSCy54eWehBJyZdG2iE84w
1,qVc8ODYU5SZjKXVBgXdI7w,pe42u7DcCH2QmI81NX-8qA
2,qVc8ODYU5SZjKXVBgXdI7w,EjlCGf14tYMPJ0rsrL703w
3,qVc8ODYU5SZjKXVBgXdI7w,7OagHIAKx2Rm9z3CQ0OmvA
4,qVc8ODYU5SZjKXVBgXdI7w,uZoFvKs0dahfffIqHQf1eA


In [8]:
friend_df.info()
friend_df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106104025 entries, 0 to 106104024
Data columns (total 2 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   user_id    object
 1   friend_id  object
dtypes: object(2)
memory usage: 1.6+ GB


(106104025, 2)

In [11]:
# check if friend_id is the user_id of others to understand the meaning

unique_user_ids = set(friend_df['user_id'])
unique_friend_ids = set(friend_df['friend_id'])

# check if friend_id is an user_id
friends_as_users = unique_friend_ids.intersection(unique_user_ids)

# If friends_as_users is not empty, then some friend_id is also user_id
if friends_as_users:
    print("Some friend_ids are also user_ids")
else:
    print("No friend_ids are user_ids.")

Some friend_ids are also user_ids


In [12]:
# remove "friends" from user_df
user_df = user_df.drop('friends', axis=1)
user_df.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,fans,average_stars,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,267,3.91,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...",3138,3.74,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013",52,3.32,...,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"2009,2010,2011",28,4.27,...,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,,1,3.54,...,1,0,0,0,1,1,0,0,0,0


## eliteyear_df -- from user_df (v)

In [13]:
# Function to explode the 'elite' column into multiple rows
def explode_elite(df, user_id_col, elite_col):
    # Create a new DataFrame for the elite years
    elite_df = df[elite_col].str.split(',', expand=True).stack().reset_index(level=1, drop=True)
    elite_df.name = 'elite_year'  # Rename the series to 'elite_year'
    
    # Combine the elite years with the user IDs
    combined_df = df[[user_id_col]].join(elite_df)
    
    return combined_df.reset_index(drop=True)

# Apply the function to user_df to create eliteyear_df
eliteyear_df = explode_elite(user_df, 'user_id', 'elite')

In [14]:
eliteyear_df.head(10)

,user_id,elite_year
0,qVc8ODYU5SZjKXVBgXdI7w,2007
1,j14WgRoU_-2ZE1aw1dXrJg,2009
2,j14WgRoU_-2ZE1aw1dXrJg,2010
3,j14WgRoU_-2ZE1aw1dXrJg,2011
4,j14WgRoU_-2ZE1aw1dXrJg,2012
5,j14WgRoU_-2ZE1aw1dXrJg,2013
6,j14WgRoU_-2ZE1aw1dXrJg,2014
7,j14WgRoU_-2ZE1aw1dXrJg,2015
8,j14WgRoU_-2ZE1aw1dXrJg,2016
9,j14WgRoU_-2ZE1aw1dXrJg,2017


In [15]:
eliteyear_df.info()
eliteyear_df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2277195 entries, 0 to 2277194
Data columns (total 2 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   user_id     object
 1   elite_year  object
dtypes: object(2)
memory usage: 34.7+ MB


(2277195, 2)

In [16]:
# remove "elite" from user_df
user_df = user_df.drop('elite', axis=1)
user_df.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,267,3.91,250,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,3138,3.74,1145,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,52,3.32,89,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,28,4.27,24,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,1,3.54,1,1,0,0,0,1,1,0,0,0,0


# Load Datas

In [8]:
!pip install pymysql
!pip install pandas sqlalchemy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.4 MB/s eta 0:00:00


In [9]:
#3
import pymysql
import sqlalchemy
from sqlalchemy import create_engine

username = 'team12'  # Replace with your username
password = 'team12'  # Replace with your password
host = 'testproject.mysql.database.azure.com'  # Replace with your host
database = 'team12'  # Replace with your database name
connection_string = f"mysql+pymysql://{username}:{password}@{host}/{database}"

ssl_args = {
            'ssl_ca': 'path_to/ca-cert.pem',
            'ssl_cert': 'path_to/client-cert.pem',
            'ssl_key': 'path_to/client-key.pem'
}

engine = create_engine(
    connection_string,
    connect_args={'ssl': ssl_args})

## business (v)

In [37]:
# check to set proper metadata
business_df.head()
business_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   150346 non-null  object 
 1   name          150346 non-null  object 
 2   address       150346 non-null  object 
 3   city          150346 non-null  object 
 4   state         150346 non-null  object 
 5   postal_code   150346 non-null  object 
 6   latitude      150346 non-null  float64
 7   longitude     150346 non-null  float64
 8   stars         150346 non-null  float64
 9   review_count  150346 non-null  int64  
 10  is_open       150346 non-null  int64  
dtypes: float64(3), int64(2), object(6)
memory usage: 12.6+ MB


In [44]:
#load business
from sqlalchemy import create_engine, MetaData, Table, Column, String, Float, Integer

# Define database metadata
metadata = MetaData()

business_table = Table('business', metadata,
                       Column('business_id', String(255), primary_key=True),
                       Column('name', String(255)),
                       Column('address', String(255)),
                       Column('city', String(255)),
                       Column('state', String(255)),
                       Column('postal_code', String(255)),
                       Column('latitude', Float),
                       Column('longitude', Float),
                       Column('stars', Float),
                       Column('review_count', Integer),
                       Column('is_open', Integer))

# Create a table in the database if it does not already exist
metadata.create_all(engine)

# Upload
batch_size = 50000
for start in range(0, business_df.shape[0], batch_size):
    df_subset = business_df.iloc[start:start + batch_size]
    if_exists_param = 'replace' if start == 0 else 'append'
    df_subset.to_sql(name="business", con=engine, if_exists=if_exists_param, index=False, chunksize=50000, dtype={
        'business_id': String(255),
        'name': String(255),
        'address': String(255),
        'city': String(255),
        'state': String(255),
        'postal_code': String(255),
        'latitude': Float,
        'longitude': Float,
        'stars': Float,
        'review_count': Integer,
        'is_open': Integer
    })
    print(f"Uploaded batch {start // batch_size + 1}")

Uploaded batch 1
Uploaded batch 2
Uploaded batch 3
Uploaded batch 4


## attributes

In [39]:
# check to set proper metadata
attributes_df.head()

,business_id,ByAppointmentOnly,BusinessAcceptsCreditCards,BikeParking,RestaurantsPriceRange2,CoatCheck,RestaurantsTakeOut,RestaurantsDelivery,Caters,WiFi,...,AcceptsInsurance,BestNights,BYOB,Corkage,BYOBCorkage,HairSpecializesIn,Open24Hours,RestaurantsCounterService,AgesAllowed,DietaryRestrictions
0,Pns2l4eNsfO8kk83dixA6A,True,False,False,NaN,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,mpf3x-BjTdTEA3yCZrAYPw,False,True,False,NaN,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,tUFrWirKiKi_TAnsVWINQQ,False,True,True,2.0,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,MTSW4McQd7CbVtyjqoe9mw,False,False,True,1.0,False,True,False,True,True,...,False,False,False,False,False,False,False,False,False,False
4,mWMc6_wTdE0EUBKIGXDVfA,False,True,True,NaN,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [40]:
# check to set proper metadata
attributes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 44 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   business_id                 150346 non-null  object 
 1   ByAppointmentOnly           150346 non-null  object 
 2   BusinessAcceptsCreditCards  150346 non-null  object 
 3   BikeParking                 150346 non-null  object 
 4   RestaurantsPriceRange2      85280 non-null   float64
 5   CoatCheck                   150346 non-null  object 
 6   RestaurantsTakeOut          150346 non-null  object 
 7   RestaurantsDelivery         150346 non-null  object 
 8   Caters                      150346 non-null  object 
 9   WiFi                        150346 non-null  object 
 10  BusinessParking_garage      150346 non-null  bool   
 11  BusinessParking_street      150346 non-null  bool   
 12  BusinessParking_validated   150346 non-null  bool   
 13  BusinessParkin

In [53]:
# load attributes
from sqlalchemy.types import Float, String

# Define metadata
dtype_mapping = {
    'business_id': String(255),
    'ByAppointmentOnly': String(255),
    'BusinessAcceptsCreditCards': String(255),
    'BikeParking': String(255),
    'RestaurantsPriceRange2': Float,
    'CoatCheck': String(255),
    'RestaurantsTakeOut': String(255),
    'RestaurantsDelivery': String(255),
    'Caters': String(255),
    'WiFi': String(255),
    'BusinessParking_garage': Boolean,
    'BusinessParking_street': Boolean,
    'BusinessParking_validated': Boolean,
    'BusinessParking_lot': Boolean,
    'BusinessParking_valet': Boolean,
    'WheelchairAccessible': String(255),
    'HappyHour': String(255),
    'OutdoorSeating': String(255),
    'HasTV': String(255),
    'RestaurantsReservations': String(255),
    'DogsAllowed': String(255),
    'Alcohol': String(255),
    'GoodForKids': String(255),
    'RestaurantsAttire': Boolean,
    'Ambience': String(255),
    'RestaurantsTableService': String(255),
    'RestaurantsGoodForGroups': String(255),
    'DriveThru': String(255),
    'NoiseLevel': String(255),
    'GoodForMeal': String(255),  
    'BusinessAcceptsBitcoin': String(255),
    'Smoking': String(255),
    'Music': String(255),  
    'GoodForDancing': Boolean,
    'AcceptsInsurance': String(255),
    'BestNights': String(255),  
    'BYOB': String(255),
    'Corkage': String(255),
    'BYOBCorkage': String(255),
    'HairSpecializesIn': String(255),  
    'Open24Hours': Boolean,
    'RestaurantsCounterService': Boolean,
    'AgesAllowed': String(255),
    'DietaryRestrictions': String(255)  
}

# load with batches
batch_size = 50000
for start in range(0, attributes_df.shape[0], batch_size):
    df_subset = attributes_df.iloc[start:start + batch_size]
    if_exists_param = 'replace' if start == 0 else 'append'
    df_subset.to_sql(name="attributes", con=engine, if_exists=if_exists_param, index=False, chunksize=50000, dtype=dtype_mapping)
    print(f"Uploaded batch {start // batch_size + 1}")

Uploaded batch 1
Uploaded batch 2
Uploaded batch 3
Uploaded batch 4


## categories (v)

In [47]:
# check to set proper metadata
categories_df.head(20)

,business_id,cat_id,category
0,Pns2l4eNsfO8kk83dixA6A,1,Doctors
1,Pns2l4eNsfO8kk83dixA6A,2,Traditional Chinese Medicine
2,Pns2l4eNsfO8kk83dixA6A,3,Naturopathic/Holistic
3,Pns2l4eNsfO8kk83dixA6A,4,Acupuncture
4,Pns2l4eNsfO8kk83dixA6A,5,Health & Medical
5,Pns2l4eNsfO8kk83dixA6A,6,Nutritionists
6,mpf3x-BjTdTEA3yCZrAYPw,7,Shipping Centers
7,mpf3x-BjTdTEA3yCZrAYPw,8,Local Services
8,mpf3x-BjTdTEA3yCZrAYPw,9,Notaries
9,mpf3x-BjTdTEA3yCZrAYPw,10,Mailbox Centers


In [46]:
# check to set proper metadata
categories_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 668695 entries, 0 to 668694
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   business_id  668695 non-null  object
 1   cat_id       668695 non-null  int64 
 2   category     668695 non-null  object
dtypes: int64(1), object(2)
memory usage: 15.3+ MB


In [57]:
# load categories
from sqlalchemy import Integer, String

# Define metadata
dtype_mapping = {
    'business_id': String(255),
    'cat_id': Integer,
    'category': String(255)
}

# load with batches
batch_size = 50000
for start in range(0, categories_df.shape[0], batch_size):
    df_subset = categories_df.iloc[start:start + batch_size]
    if_exists_param = 'replace' if start == 0 else 'append'
    df_subset.to_sql(name="categories", con=engine, if_exists=if_exists_param, index=False, chunksize=50000, dtype=dtype_mapping)
    print(f"Uploaded batch {start // batch_size + 1}")

Uploaded batch 1
Uploaded batch 2
Uploaded batch 3
Uploaded batch 4
Uploaded batch 5
Uploaded batch 6
Uploaded batch 7
Uploaded batch 8
Uploaded batch 9
Uploaded batch 10
Uploaded batch 11
Uploaded batch 12
Uploaded batch 13
Uploaded batch 14


## hours (v)

In [54]:
# check to set proper metadata
hours_df.head()

,business_id,day_of_week,start_hour,end_hour,duration
0,mpf3x-BjTdTEA3yCZrAYPw,Monday,0:0,0:0,0:00
1,mpf3x-BjTdTEA3yCZrAYPw,Tuesday,8:0,18:30,10:30
2,mpf3x-BjTdTEA3yCZrAYPw,Wednesday,8:0,18:30,10:30
3,mpf3x-BjTdTEA3yCZrAYPw,Thursday,8:0,18:30,10:30
4,mpf3x-BjTdTEA3yCZrAYPw,Friday,8:0,18:30,10:30


In [55]:
# check to set proper metadata
hours_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801015 entries, 0 to 801014
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   business_id  801015 non-null  object
 1   day_of_week  801015 non-null  object
 2   start_hour   801015 non-null  object
 3   end_hour     801015 non-null  object
 4   duration     801015 non-null  object
dtypes: object(5)
memory usage: 30.6+ MB


In [60]:
# load hours
from sqlalchemy.types import String

# Define metadata
dtype_mapping = {
    'business_id': String(255),
    'day_of_week': String(50),
    'start_hour': String(50),
    'end_hour': String(50),
    'duration': String(50)
}

# load with batches
batch_size = 50000
for start in range(0, hours_df.shape[0], batch_size):
    df_subset = hours_df.iloc[start:start + batch_size]
    if_exists_param = 'replace' if start == 0 else 'append'
    df_subset.to_sql(name="hours", con=engine, if_exists=if_exists_param, index=False, chunksize=50000, dtype=dtype_mapping)
    print(f"Uploaded batch {start // batch_size + 1}")
print(end)

Uploaded batch 1
Uploaded batch 2
Uploaded batch 3
Uploaded batch 4
Uploaded batch 5
Uploaded batch 6
Uploaded batch 7
Uploaded batch 8
Uploaded batch 9
Uploaded batch 10
Uploaded batch 11
Uploaded batch 12
Uploaded batch 13
Uploaded batch 14
Uploaded batch 15
Uploaded batch 16
Uploaded batch 17
19:0


## checkin (v)

In [58]:
# check to set proper metadata
checkin_df_split.head(20)

,business_id,date,year,month,day_of_week,hour_of_day,week_of_year,is_weekend,checkin_count,unique_days_count,avg_checkins_per_day
0,---kPU91CF4Lq2-WlRu9Lw,2020-03-13 21:10:56,2020,3,Friday,21,11,False,11,11,1.0
1,---kPU91CF4Lq2-WlRu9Lw,2020-06-02 22:18:06,2020,6,Tuesday,22,23,False,11,11,1.0
2,---kPU91CF4Lq2-WlRu9Lw,2020-07-24 22:42:27,2020,7,Friday,22,30,False,11,11,1.0
3,---kPU91CF4Lq2-WlRu9Lw,2020-10-24 21:36:13,2020,10,Saturday,21,43,True,11,11,1.0
4,---kPU91CF4Lq2-WlRu9Lw,2020-12-09 21:23:33,2020,12,Wednesday,21,50,False,11,11,1.0
5,---kPU91CF4Lq2-WlRu9Lw,2021-01-20 17:34:57,2021,1,Wednesday,17,3,False,11,11,1.0
6,---kPU91CF4Lq2-WlRu9Lw,2021-04-30 21:02:03,2021,4,Friday,21,17,False,11,11,1.0
7,---kPU91CF4Lq2-WlRu9Lw,2021-05-25 21:16:54,2021,5,Tuesday,21,21,False,11,11,1.0
8,---kPU91CF4Lq2-WlRu9Lw,2021-08-06 21:08:08,2021,8,Friday,21,31,False,11,11,1.0
9,---kPU91CF4Lq2-WlRu9Lw,2021-10-02 15:15:42,2021,10,Saturday,15,39,True,11,11,1.0


In [59]:
# check to set proper metadata
checkin_df_split.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13356875 entries, 0 to 13356874
Data columns (total 11 columns):
 #   Column                Dtype         
---  ------                -----         
 0   business_id           object        
 1   date                  datetime64[ns]
 2   year                  int32         
 3   month                 int32         
 4   day_of_week           object        
 5   hour_of_day           int32         
 6   week_of_year          UInt32        
 7   is_weekend            bool          
 8   checkin_count         int64         
 9   unique_days_count     int64         
 10  avg_checkins_per_day  float64       
dtypes: UInt32(1), bool(1), datetime64[ns](1), float64(1), int32(3), int64(2), object(2)
memory usage: 840.7+ MB


In [64]:
#load checkin; in batches to track the process due to too much lines(13356875, 2)

from sqlalchemy.types import DateTime, Float, Integer, String, Boolean

# Define metadata
dtype_mapping = {
    'business_id': String(255),
    'date': DateTime,
    'year': Integer,
    'month': Integer,
    'day_of_week': String(50),
    'hour_of_day': Integer,
    'week_of_year': Integer,
    'is_weekend': Boolean,
    'checkin_count': Integer,
    'unique_days_count': Integer,
    'avg_checkins_per_day': Float
}

# load with batches
batch_size = 50000
for start in range(0, checkin_df_split.shape[0], batch_size):
    df_subset = checkin_df_split.iloc[start:start + batch_size]
    if_exists_param = 'replace' if start == 0 else 'append'
    df_subset.to_sql(name="checkin", con=engine, if_exists=if_exists_param, index=False, chunksize=50000, dtype=dtype_mapping)
    print(f"Uploaded batch {start // batch_size + 1}")
print("end")

Uploaded batch 1
Uploaded batch 2
Uploaded batch 3
Uploaded batch 4
Uploaded batch 5
Uploaded batch 6
Uploaded batch 7
Uploaded batch 8
Uploaded batch 9
Uploaded batch 10
Uploaded batch 11
Uploaded batch 12
Uploaded batch 13
Uploaded batch 14
Uploaded batch 15
Uploaded batch 16
Uploaded batch 17
Uploaded batch 18
Uploaded batch 19
Uploaded batch 20
Uploaded batch 21
Uploaded batch 22
Uploaded batch 23
Uploaded batch 24
Uploaded batch 25
Uploaded batch 26
Uploaded batch 27
Uploaded batch 28
Uploaded batch 29
Uploaded batch 30
Uploaded batch 31
Uploaded batch 32
Uploaded batch 33
Uploaded batch 34
Uploaded batch 35
Uploaded batch 36
Uploaded batch 37
Uploaded batch 38
Uploaded batch 39
Uploaded batch 40
Uploaded batch 41
Uploaded batch 42
Uploaded batch 43
Uploaded batch 44
Uploaded batch 45
Uploaded batch 46
Uploaded batch 47
Uploaded batch 48
Uploaded batch 49
Uploaded batch 50
Uploaded batch 51
Uploaded batch 52
Uploaded batch 53
Uploaded batch 54
Uploaded batch 55
Uploaded batch 56
U

## review (v)

In [61]:
# check to set proper metadata
review_df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [63]:
# check to set proper metadata
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6990280 entries, 0 to 6990279
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   review_id    object        
 1   user_id      object        
 2   business_id  object        
 3   stars        float64       
 4   useful       int64         
 5   funny        int64         
 6   cool         int64         
 7   text         object        
 8   date         datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 480.0+ MB


In [70]:
#load review; in batches to track the process due to too much lines(6990280, 9)

from sqlalchemy import create_engine, MetaData, Table, Column
from sqlalchemy.types import DateTime, Float, Integer, String, Text

# Define metadata
metadata = MetaData()

review_table = Table(
    'review', metadata,
    Column('review_id', String(255), primary_key=True),
    Column('user_id', String(255)),
    Column('business_id', String(255)),
    Column('stars', Float),
    Column('useful', Integer),
    Column('funny', Integer),
    Column('cool', Integer),
    Column('text', Text),
    Column('date', DateTime)
)

review_table.create(bind=engine, checkfirst=True)

# load with batches
batch_size = 500000
for start in range(0, review_df.shape[0], batch_size):
    df_subset = review_df.iloc[start:start + batch_size]
    if_exists_param = 'replace' if start == 0 else 'append'
    df_subset.to_sql(name=review_table.name, con=engine, if_exists=if_exists_param, index=False, chunksize=50000, dtype=dtype_mapping)
    print(f"Uploaded batch {start // batch_size + 1}")

print("end")

Uploaded batch 1
Uploaded batch 2
Uploaded batch 3
Uploaded batch 4
Uploaded batch 5
Uploaded batch 6
Uploaded batch 7
Uploaded batch 8
Uploaded batch 9
Uploaded batch 10
Uploaded batch 11
Uploaded batch 12
Uploaded batch 13
Uploaded batch 14
end


## tip (v)

In [65]:
# check to set proper metadata
tip_df.head()

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0


In [68]:
# check to set proper metadata
tip_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908915 entries, 0 to 908914
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   user_id           908915 non-null  object        
 1   business_id       908915 non-null  object        
 2   text              908915 non-null  object        
 3   date              908915 non-null  datetime64[ns]
 4   compliment_count  908915 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 34.7+ MB


In [71]:
# load tip
# tip_df.to_sql(name = "tip", con = engine, if_exists='replace', index=True, chunksize=50000) 

from sqlalchemy.types import DateTime, Integer, String, Text

# Define metadata
dtype_mapping = {
    'user_id': String(255),
    'business_id': String(255),
    'text': Text,
    'date': DateTime,
    'compliment_count': Integer
}

# load with batches
batch_size = 100000
for start in range(0, tip_df.shape[0], batch_size):
    df_subset = tip_df.iloc[start:start + batch_size]
    if_exists_param = 'replace' if start == 0 else 'append'
    df_subset.to_sql(name="tip", con=engine, if_exists=if_exists_param, index=False, chunksize=50000, dtype=dtype_mapping)
    print(f"Uploaded batch {start // batch_size + 1}")
print("end")

Uploaded batch 1
Uploaded batch 2
Uploaded batch 3
Uploaded batch 4
Uploaded batch 5
Uploaded batch 6
Uploaded batch 7
Uploaded batch 8
Uploaded batch 9
Uploaded batch 10
end


## user (v)

In [17]:
# check to set proper metadata
user_df.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,267,3.91,250,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,3138,3.74,1145,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,52,3.32,89,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,28,4.27,24,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,1,3.54,1,1,0,0,0,1,1,0,0,0,0


In [18]:
# check to set proper metadata
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1987897 entries, 0 to 1987896
Data columns (total 20 columns):
 #   Column              Dtype  
---  ------              -----  
 0   user_id             object 
 1   name                object 
 2   review_count        int64  
 3   yelping_since       object 
 4   useful              int64  
 5   funny               int64  
 6   cool                int64  
 7   fans                int64  
 8   average_stars       float64
 9   compliment_hot      int64  
 10  compliment_more     int64  
 11  compliment_profile  int64  
 12  compliment_cute     int64  
 13  compliment_list     int64  
 14  compliment_note     int64  
 15  compliment_plain    int64  
 16  compliment_cool     int64  
 17  compliment_funny    int64  
 18  compliment_writer   int64  
 19  compliment_photos   int64  
dtypes: float64(1), int64(16), object(3)
memory usage: 303.3+ MB


In [26]:
#load user; in batches to track the process due to too much lines(1987897, 22)

from sqlalchemy import create_engine, MetaData, Table, Column
from sqlalchemy.types import DateTime, Float, Integer, String

# Define metadata
metadata = MetaData()

user_table = Table(
    'user', metadata,
    Column('user_id', String(255), primary_key=True),
    Column('name', String(255)),
    Column('review_count', Integer),
    Column('yelping_since', DateTime),
    Column('useful', Integer),
    Column('funny', Integer),
    Column('cool', Integer),
    Column('fans', Integer),
    Column('average_stars', Float),
    Column('compliment_hot', Integer),
    Column('compliment_more', Integer),
    Column('compliment_profile', Integer),
    Column('compliment_cute', Integer),
    Column('compliment_list', Integer),
    Column('compliment_note', Integer),
    Column('compliment_plain', Integer),
    Column('compliment_cool', Integer),
    Column('compliment_funny', Integer),
    Column('compliment_writer', Integer),
    Column('compliment_photos', Integer)
)

user_table.create(bind=engine, checkfirst=True)

user_df['yelping_since'] = pd.to_datetime(user_df['yelping_since'])

# load with batches
batch_size = 500000
for start in range(0, user_df.shape[0], batch_size):
    df_subset = user_df.iloc[start:start + batch_size]
    if_exists_param = 'replace' if start == 0 else 'append'
    df_subset.to_sql(name=user_table.name, con=engine, if_exists=if_exists_param, index=False, chunksize=50000)
    print(f"Uploaded batch {start // batch_size + 1}")
print("end")

Uploaded batch 1
Uploaded batch 2
Uploaded batch 3
Uploaded batch 4
end


## friends

In [5]:
# check to set proper metadata
friend_df.head()

,user_id,friend_id
0,qVc8ODYU5SZjKXVBgXdI7w,NSCy54eWehBJyZdG2iE84w
1,qVc8ODYU5SZjKXVBgXdI7w,pe42u7DcCH2QmI81NX-8qA
2,qVc8ODYU5SZjKXVBgXdI7w,EjlCGf14tYMPJ0rsrL703w
3,qVc8ODYU5SZjKXVBgXdI7w,7OagHIAKx2Rm9z3CQ0OmvA
4,qVc8ODYU5SZjKXVBgXdI7w,uZoFvKs0dahfffIqHQf1eA


In [6]:
# check to set proper metadata
friend_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106104025 entries, 0 to 106104024
Data columns (total 2 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   user_id    object
 1   friend_id  object
dtypes: object(2)
memory usage: 1.6+ GB


In [10]:
#load friend

from sqlalchemy import create_engine, MetaData, Table, Column
from sqlalchemy.types import String

# Define metadata
metadata = MetaData()

friends_table = Table(
    'friends', metadata,
    Column('user_id', String(255)),
    Column('friend_id', String(255))
)

friends_table.create(bind=engine, checkfirst=True)

# load with batches
batch_size = 500000  # choose an appropriate batch size due to huge amount of data, 
for start in range(0, friend_df.shape[0], batch_size):
    df_subset = friend_df.iloc[start:start + batch_size]
    if_exists_param = 'replace' if start == 0 else 'append'
    df_subset.to_sql(name=friends_table.name, con=engine, if_exists=if_exists_param, index=False, chunksize=50000)
    print(f"Uploaded batch {start // batch_size + 1}")
print("end")

Uploaded batch 1
Uploaded batch 2
Uploaded batch 3
Uploaded batch 4
Uploaded batch 5
Uploaded batch 6
Uploaded batch 7
Uploaded batch 8
Uploaded batch 9
Uploaded batch 10
Uploaded batch 11
Uploaded batch 12
Uploaded batch 13
Uploaded batch 14
Uploaded batch 15
Uploaded batch 16
Uploaded batch 17
Uploaded batch 18
Uploaded batch 19
Uploaded batch 20
Uploaded batch 21
Uploaded batch 22
Uploaded batch 23
Uploaded batch 24
Uploaded batch 25
Uploaded batch 26
Uploaded batch 27
Uploaded batch 28
Uploaded batch 29
Uploaded batch 30
Uploaded batch 31
Uploaded batch 32
Uploaded batch 33
Uploaded batch 34
Uploaded batch 35
Uploaded batch 36
Uploaded batch 37
Uploaded batch 38
Uploaded batch 39
Uploaded batch 40
Uploaded batch 41
Uploaded batch 42
Uploaded batch 43
Uploaded batch 44
Uploaded batch 45
Uploaded batch 46
Uploaded batch 47
Uploaded batch 48
Uploaded batch 49
Uploaded batch 50
Uploaded batch 51
Uploaded batch 52
Uploaded batch 53
Uploaded batch 54
Uploaded batch 55
Uploaded batch 56
U

## elite (v)

In [29]:
# check to set proper metadata
eliteyear_df.head(30)

,user_id,elite_year
0,qVc8ODYU5SZjKXVBgXdI7w,2007
1,j14WgRoU_-2ZE1aw1dXrJg,2009
2,j14WgRoU_-2ZE1aw1dXrJg,2010
3,j14WgRoU_-2ZE1aw1dXrJg,2011
4,j14WgRoU_-2ZE1aw1dXrJg,2012
5,j14WgRoU_-2ZE1aw1dXrJg,2013
6,j14WgRoU_-2ZE1aw1dXrJg,2014
7,j14WgRoU_-2ZE1aw1dXrJg,2015
8,j14WgRoU_-2ZE1aw1dXrJg,2016
9,j14WgRoU_-2ZE1aw1dXrJg,2017


In [22]:
# check to set proper metadata
eliteyear_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2277195 entries, 0 to 2277194
Data columns (total 2 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   user_id     object
 1   elite_year  object
dtypes: object(2)
memory usage: 34.7+ MB


In [31]:
# load elite
from sqlalchemy import create_engine, MetaData, Table, Column
from sqlalchemy.types import Integer, String

# Define metadata
metadata = MetaData()

elite_table = Table(
    'elite', metadata,
    Column('user_id', String(255)),
    Column('elite_year', String(255))
)

elite_table.create(bind=engine, checkfirst=True)

# load with batches
batch_size = 100000
for start in range(0, eliteyear_df.shape[0], batch_size):
    df_subset = eliteyear_df.iloc[start:start + batch_size]
    if_exists_param = 'replace' if start == 0 else 'append'
    df_subset.to_sql(name=elite_table.name, con=engine, if_exists=if_exists_param, index=False, chunksize=50000)
    print(f"Uploaded batch {start // batch_size + 1}")
print("end")

Uploaded batch 1
Uploaded batch 2
Uploaded batch 3
Uploaded batch 4
Uploaded batch 5
Uploaded batch 6
Uploaded batch 7
Uploaded batch 8
Uploaded batch 9
Uploaded batch 10
Uploaded batch 11
Uploaded batch 12
Uploaded batch 13
Uploaded batch 14
Uploaded batch 15
Uploaded batch 16
Uploaded batch 17
Uploaded batch 18
Uploaded batch 19
Uploaded batch 20
Uploaded batch 21
Uploaded batch 22
Uploaded batch 23
end
